<a href="https://colab.research.google.com/github/dae1jeong/SSU_25_NLP_project/blob/main/notice%20crawling/Soongsil_University_Notice_Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime

# 숭실대학교 스캐치(Scatch) 공지사항 페이지 URL을 설정합니다.
BASE_URL = "https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec%82%ac%ed%95%ad/"
MAX_PAGES = 80  # 최대 크롤링 페이지 수를 설정
TARGET_YEAR = 2025 # 2025년 이후 공지사항만 추출

def fetch_notices_all_pages(base_url, max_pages):
    """
    여러 페이지를 순회하며 공지사항 목록을 가져오고 2025년 이후의 항목만 필터링합니다.
    """
    all_notices_data = []
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    crawled_until_max = True

    for page_num in range(1, max_pages + 1):

        # URL 경로 설정: '/page/N/' 형태
        if page_num == 1:
            url = base_url
        else:
            url = f"{base_url}page/{page_num}/"

        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')

            notice_items = soup.find_all('div', class_=lambda c: c and 'row no-gutters align-items-center' in c)

            # ----------------------------------------------------
            # 📢 페이지별 완료 메시지 출력 로직 (수정) 📢
            # ----------------------------------------------------
            # 항목을 찾기 전에, 페이지 요청이 성공하면 바로 출력
            print(f"✅ 페이지 {page_num} 크롤링 완료. 다음 페이지로 진행합니다.")
            # ----------------------------------------------------

            if not notice_items:
                print(f"✅ 페이지 {page_num} ({url})에서 더 이상 공지사항 항목을 찾을 수 없어 크롤링을 종료합니다.")
                crawled_until_max = False
                break

            # 페이지 내 항목 수집 및 효율화된 필터링 로직 (유지)
            for item in notice_items:
                date_element = item.select_one('.notice_col1 .h2')
                date = date_element.text.strip() if date_element else None

                if date:
                    try:
                        year = int(date[0:4])
                        if year < TARGET_YEAR:
                            continue
                    except ValueError:
                        continue
                else:
                    continue

                # 2025년 이후 항목에 대해서만 나머지 정보 추출
                try:
                    status_element = item.select_one('.notice_col2 .tag')
                    status = status_element.text.strip() if status_element else 'N/A'

                    link_element = item.select_one('.notice_col3 a')
                    if link_element:
                        link_href = link_element['href'].strip()
                        category_element = link_element.select_one('.label')
                        category = category_element.text.strip() if category_element else 'N/A'
                        title_span = link_element.select_one('span[class*="d-inline-b"].m-pt-5')
                        title = title_span.text.strip() if title_span else '제목 없음 (파싱 오류)'
                    else:
                        link_href, category, title = 'N/A', 'N/A', 'N/A'

                    department_element = item.select_one('.notice_col4')
                    department = department_element.text.strip() if department_element else 'N/A'

                    views_element = item.select_one('.notice_col5')
                    views = views_element.text.strip() if views_element else 'N/A'

                    all_notices_data.append({
                        '작성일': date,
                        '진행상황': status,
                        '카테고리': category,
                        '제목': title,
                        '링크': link_href,
                        '등록부서': department,
                        '조회수': views
                    })

                except Exception as e:
                    continue

        except requests.exceptions.RequestException as e:
            print(f"❌ 페이지 {page_num} ({url})를 가져오는 중 오류가 발생했습니다: {e}")
            crawled_until_max = False
            break

    # 최대 설정된 페이지 수에 도달한 경우에만 메시지 출력
    if crawled_until_max and page_num == max_pages:
         print(f"🚨 최대 설정된 {max_pages} 페이지까지 크롤링을 완료했습니다.")

    return all_notices_data

# 기존 save_to_csv 함수는 동일하게 사용합니다.
def save_to_csv(data, filename="ssu_notices_2025_onwards.csv"):
    if not data:
        print("저장할 데이터가 없습니다.")
        return

    fieldnames = ['작성일', '진행상황', '카테고리', '제목', '링크', '등록부서', '조회수']

    try:
        with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(data)

        print(f"\n데이터가 '{filename}' 파일에 성공적으로 저장되었습니다.")
    except Exception as e:
        print(f"\nCSV 저장 중 오류가 발생했습니다: {e}")

if __name__ == '__main__':
    print(f"--- 숭실대학교 공지사항 크롤링 시작 ({TARGET_YEAR}년 이후 필터링 및 페이지네이션) ---")

    results = fetch_notices_all_pages(BASE_URL, MAX_PAGES)

    if results:
        print(f"\n✅ 최종적으로 총 {len(results)}개의 ({TARGET_YEAR}년 이후) 공지사항을 찾았습니다.")

        for i, notice in enumerate(results[:5]):
            print(f"--- 항목 {i+1} ---")
            print(f"작성일: {notice['작성일']}, 제목: {notice['제목']}")

        if len(results) > 5:
            print(f"...\n총 {len(results)}개 중 상위 5개 출력 완료.")

        save_to_csv(results)

    else:
        print("공지사항을 찾지 못했거나 크롤링에 실패했습니다. URL과 CSS Selector를 다시 확인해 주세요.")

--- 숭실대학교 공지사항 크롤링 시작 (2025년 이후 필터링 및 페이지네이션) ---
✅ 페이지 1 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 2 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 3 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 4 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 5 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 6 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 7 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 8 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 9 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 10 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 11 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 12 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 13 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 14 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 15 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 16 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 17 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 18 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 19 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 20 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 21 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 22 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 23 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 24 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 25 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 26 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 27 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 28 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 29 크롤링 완료. 다음 페이지로 진행합니다.
✅ 페이지 30 크롤링 완료. 다음 페이지로 진행합니다.

In [10]:
import pandas as pd

df = pd.read_csv('/content/ssu_notices_2025_onwards.csv')

df.tail(15)

,작성일,진행상황,카테고리,제목,링크,등록부서,조회수
1095,2025.01.06,완료,장학,2024학년도 2학기 특별장학금(가계곤란) 선발 안내,https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec...,장학팀,19130
1096,2025.01.06,완료,장학,"2024학년도 2학기 특별장학금(주거비, 교통비 및 근면숭실) 선발 안내",https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec...,장학팀,16231
1097,2025.01.06,NaN,봉사,2025년 서울런 멘토단 참여 대학생 모집,https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec...,사회공헌팀,30876
1098,2025.01.06,완료,교원채용,"2025학년도 1학기 사회과학대학 강사 및 겸임교수 채용 공고(사회복지학부, 평생교...",https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec...,사회과학대학 교학팀,22549
1099,2025.01.03,NaN,학사,2025-1학기 현장실습학기제 학생 참여 안내,https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec...,현장실습지원센터,70427
1100,2025.01.03,완료,장학,2025학년도 1학기 한국장학재단 학자금대출 안내,https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec...,장학팀,25508
1101,2025.01.03,완료,장학,2025 KB라이프 장학생 모집,https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec...,장학팀,18749
1102,2025.01.03,완료,장학,한국장학재단 학자금대출 특별승인제도 안내,https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec...,장학팀,17273
1103,2025.01.03,완료,장학,★2025학년도 1학기 재단법인 해성문화재단 장학생 선발,https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec...,장학팀,17332
1104,2025.01.03,완료,교원채용,2025학년도 1학기 숭실대학교 삼성반도체겸임교수 채용 공고,https://scatch.ssu.ac.kr/%ea%b3%b5%ec%a7%80%ec...,교무팀,24385
